In [ ]:
!pip -q install faiss-cpu sentence-transformers numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.1 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
import numpy as np

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/first_model_data1.csv")

faq_questions = df["Question"].tolist()
faq_answers = df["Answer"].tolist()

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

def convert_q_to_embeddings(questions):
  embeddings = model.encode(faq_questions)

  return embeddings

faq_embeddings = convert_q_to_embeddings(faq_questions)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/952 [00:00<?, ?it/s]

In [ ]:
faq_embeddings = np.array(faq_embeddings, dtype="float32")

index = faiss.IndexFlatL2(faq_embeddings.shape[1])
index.add(faq_embeddings)

In [ ]:
def search_faq(query, top_k=3):
  query_embedding = model.encode([query])
  query_embedding = np.array(query_embedding, dtype="float32")

  distances, indices = index.search(query_embedding, top_k)

  results = []

  for i in range(top_k):
    results.append({
        "question": faq_questions[indices[0][i]],
        "answer": faq_answers[indices[0][i]],
        "score": distances[0][i]
    })

  return results

In [ ]:
query = "how to workout at home?"
results = search_faq(query)

for res in results:
  print(f"Q: {res['question']}")
  print(f"A: {res['answer']}")
  print(f"Score: {res['score']}\n")



Q: i, like most runners, tend to focus more on training and forget that some strength exercises are required to reach your performance peak. what are some good exercises that you can do at home without gym equipment?
A: to me core straining are very central. your core muscles includes all the main muscle groups in you torso and increasing the strength of these will not only help with your running but also with all sorts of back problems if you have a desk job like me. see about.com for a pretty good introduction to core training. very few core exercises requires any equipment.
Score: 0.6838144063949585

Q: i have recently started lifting at home, i have some weights, a bench press setup, a barbell and two dumbbells. i would like to include deadlifts in my workout but i feel like i am not doing it with correct form. what can i do at home to learn how to deadlift and ensure my form is good and proper and to improve it if it is not? i realize that a personal trainer would probably be the 

In [ ]:
faiss.write_index(index, "/content/drive/MyDrive/faq_fiass_index.bin")